In [ ]:
Reference : 
     * 'https://www.kaggle.com/gaborfodor/black-white-cnn-animals'
        
Class tutorial : 
    * 'https://en.wikibooks.org/wiki/A_Beginner%27s_Python_Tutorial/Classes'
    * 'https://wikidocs.net/28'

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import ast

## Importing data

First check what we have in train_simplified.zip file

In [ ]:
files = os.listdir('../input/train_simplified/')
files

Will use only the penguin drawings

In [ ]:
pg = pd.read_csv('../input/train_simplified/penguin.csv')
pg.head()

## Drawing Penguins

In [ ]:
# Take only recognizable drawings 
df = pg[pg.recognized]

In [ ]:
# timestamp
df['timestamp'] = pd.to_datetime(df.timestamp)
df.head()

In [ ]:
df = df.sort_values('timestamp', ascending = False)[-100:]
df['drawing'] = df.drawing.apply(ast.literal_eval)
df.head()

pretrial for plotting the drawings

In [ ]:
n = 3
for index, col in enumerate(df[:20].drawing):
    print('index= ', index, index//n, index%n) 
    print('col= ', col)

In [ ]:
for index, col in enumerate(df[:20].drawing):
    for a, b in col:
        print(a, -np.array(b))        

In [ ]:
# show image
n =  10   
fig, axs = plt.subplots(nrows = n, ncols = n, figsize = (10, 8))
for index, col in enumerate(df.drawing):
    ax = axs[index//n, index%n]
    for x, y in col:
        ax.plot(x,-np.array(y), lw = 3)
    ax.axis('off')
plt.show()

## Data preprocessing

In [ ]:
DP_DIR = '../input/shuffle-animal-csvs/'
INPUT_DIR = '../input/quickdraw-doodle-recognition/'
# INPUT_DIR = '../input/'
BW_DIR = '../input/black-white-cnn-animals/'
GS_DIR = '../input/greyscale-mobilenet-animals/'
BASE_SIZE = 256
NCSVS = 100

In [ ]:
def f2cat(filename: str) -> str:
    return filename.split('.')[0]

class Simplified():
    def __init__(self, input_path='./input'):
        self.input_path = input_path

    def list_all_categories(self):
        files = os.listdir(os.path.join(self.input_path, 'train_simplified'))
        return [f2cat(f) for f in files]

    def read_training_csv(self, category, nrows=None, usecols=None, drawing_transform=False):
        df = pd.read_csv(os.path.join(self.input_path, 'train_simplified', category + '.csv'),
                         nrows=nrows, parse_dates=['timestamp'], usecols=usecols)
        if drawing_transform:
            df['drawing'] = df['drawing'].apply(ast.literal_eval)
        return df

In [ ]:
s = Simplified(INPUT_DIR)

In [ ]:
animals = ['ant', 'bat', 'bear', 'bee', 'bird', 'butterfly', 'camel', 'cat', 'cow',
           'crab', 'crocodile', 'dog', 'dolphin', 'dragon', 'duck', 'elephant', 'fish',
           'flamingo', 'frog', 'giraffe', 'hedgehog', 'horse', 'kangaroo', 'lion',
           'lobster', 'monkey', 'mosquito', 'mouse', 'octopus', 'owl', 'panda',
           'parrot', 'penguin', 'pig', 'rabbit', 'raccoon', 'rhinoceros', 'scorpion',
           'sea turtle', 'shark', 'sheep', 'snail', 'snake', 'spider', 'squirrel',
           'swan', 'teddy-bear', 'tiger', 'whale', 'zebra']
NCATS = len(animals)

## Image Augmentation

## Modeling

In [ ]:
im_height = 64
im_weight = 64
im_size = (im_height, im_weight)
batch_size = 500

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense

In [ ]:
# Initialization
model = Sequential()

In [ ]:
# ConvNet_1
model.add(Conv2D(128, kernel_size = 3, input_shape = (im_height, im_weight, 3), padding = 'same', activation = 'relu'))
model.add(MaxPool2D(2, strides = 2))

# ConvNet_2
model.add(Conv2D(64, kernel_size = 3, activation = 'relu'))
model.add(MaxPool2D(2, strides = 2))

# flattening
model.add(Flatten())

# fully connected layer
model.add(Dense(output_dim = 512, activation = 'relu'))

# Reguralization
model.add(Dropout(.5))

# Softmax 
model.add(Dense(n_class, activation = 'softmax'))

# Compile
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
model.summary()

## Prediction